Imports

In [50]:
from PIL import Image
import numpy as np
from pylfsr import LFSR

## Helper Functions

- Any type to Binary Converter


In [51]:
def convertToBinary(data):
    if isinstance(data, str):
        return ''.join([ format(ord(i), "08b") for i in data ])
    elif isinstance(data, bytes) or isinstance(data, np.ndarray):
        return [ format(i, "08b") for i in data ]
    elif isinstance(data, int) or isinstance(data, np.uint8):
        return format(data, "08b")
    else:
        raise TypeError("Type not supported.")

- Binary to Decimal Converter

In [52]:
def binaryToDecimal(binary):
    decimal = 0
    for bit in binary:
        decimal = decimal*2 + int(bit)
    return decimal

- Convert image to matrix

In [53]:
def getImageMatrix(imageName):

    imageHandler = Image.open(imageName)
    pixels = imageHandler.load()
    color = 1
    if type(pixels[0,0]) == int:
        color = 0
    image_size = imageHandler.size
    
    rows = int(image_size[0])
    cols = int(image_size[1])
    image_matrix = []
    for row in range(rows):
        current_row = []
        for col in range(cols):
            current_row.append(pixels[row,col])
        image_matrix.append(current_row)
    
    return image_matrix, image_size, color


## 2D Chaotic Map

- Hennon Map (Binary)

In [54]:
def HennonMapBinary(dimension, key):

    rows = dimension[0]
    cols = dimension[1]
    x = key[0]
    y = key[1]
    a = 1.4
    b = 0.3

    # Total Number of bitSequence produced
    sequenceSize = rows * cols
    bitSequenceSize = cols
    # Each bitSequence contains k bits
    bitSequence = []       
    # Each matrix contains m*n byteArray
    matrix = []

    for i in range(sequenceSize):
        x_next = y + 1 - (a * (x**2)) 
        y_next = b * x
        x = x_next
        y = y_next
        if x <= 0.4:
            bit = 0
        else:
            bit = 1

        bitSequence.append(bit) 

        # If Bit Sequence has k bits, convert it to decimal and add it to Byte Sequence.
        if i % bitSequenceSize == bitSequenceSize - 1:
            matrix.append(bitSequence)
            bitSequence = []

    return matrix

## Decoding


In [56]:
def decode(imagename, key, poly, data_len):

    # Read the Encoded Image
    imagehandler = Image.open("images/encoded_image.png") 
    image = imagehandler.load()
    image_size = imagehandler.size

    # Read the Original Image
    imagehandler_or = Image.open(imagename) 
    image_or = imagehandler_or.load()

    # Build chaotic map and derive some available pixels
    map = HennonMapBinary(image_size, key)

    availablePixels = []
    for i in range(image_size[0]):
        for j in range(0,image_size[1]):
            if map[i][j] == 1:
                availablePixels.append([i,j,0])

    # Build LFSR
    lfsr = LFSR(initstate = [1, 1, 1, 1, 1, 1, 1, 1], fpoly=poly, counter_start_zero=True)

    data = []
    current = ""
    index = 0

    for i in range(0, data_len, 2):

        # Choose an unused pixel using LFSR from Chaotic Map

        index = (index + binaryToDecimal(lfsr.state)) % len(availablePixels)
        lfsr.next()

        while availablePixels[index][2] >= 2:
            index = (index + binaryToDecimal(lfsr.state)) % len(availablePixels)
            lfsr.next()

        row, col, usageState = availablePixels[index][0], availablePixels[index][1], availablePixels[index][2]
        
        # Extract from Red Value
        if usageState == 0:
            value = list(convertToBinary(image[row,col][0]))
            value_or = list(convertToBinary(image_or[row,col][0]))
            
            temp1 = str(int(value[-2]) ^ int(value_or[-2]))
            temp2 = str(int(value[-1]) ^ int(value_or[-1]))

        # Extract from Blue Value
        elif usageState == 1:
            value = list(convertToBinary(image[row,col][1]))
            value_or = list(convertToBinary(image_or[row,col][1]))
            
            temp1 = str(int(value[-2]) ^ int(value_or[-2]))
            temp2 = str(int(value[-1]) ^ int(value_or[-1]))
        
        # Extract from Green Value
        else:
            value = list(convertToBinary(image[row,col][2]))
            value_or = list(convertToBinary(image_or[row,col][2]))

            temp1 = str(int(value[-2]) ^ int(value_or[-2]))
            temp2 = str(int(value[-1]) ^ int(value_or[-1]))
        
        current = current + temp1 + temp2
        if len(current) == 8:
            data.append(current)
            current = ""
        availablePixels[index][2] += 1

    print(data)
    data_found = "".join(chr(int(c, 2)) for c in data)
    print(data_found)

    # Store Output in File
    filehandler = open("text\output.txt","w")
    filehandler.writelines(data_found)
    filehandler.close()

    return data_found

In [57]:
import json

def main():
    info = open('config.json')
    json_data = json.load(info)
    original_image = json_data["source"]["image_filename"]
    hennon_key = json_data["key"]["hennon_key"]
    lfsr_poly = json_data["key"]["lfsr_polynomial"]
    
    decode(original_image, hennon_key, lfsr_poly, 5880)

In [58]:
main()

['01001001', '01101110', '00100000', '01100011', '01110010', '01111001', '01110000', '01110100', '01101111', '01100111', '01110010', '01100001', '01110000', '01101000', '01111001', '00101100', '00100000', '01110000', '01101100', '01100001', '01101001', '01101110', '01110100', '01100101', '01111000', '01110100', '00100000', '01101001', '01110011', '00100000', '01110101', '01101110', '01100101', '01101110', '01100011', '01110010', '01111001', '01110000', '01110100', '01100101', '01100100', '00100000', '01100100', '01100001', '01110100', '01100001', '00100000', '01110100', '01101000', '01100001', '01110100', '00100000', '01101001', '01110011', '00100000', '01110101', '01110011', '01100101', '01100100', '00100000', '01100001', '01110011', '00100000', '01101001', '01101110', '01110000', '01110101', '01110100', '00100000', '01100110', '01101111', '01110010', '00100000', '01100101', '01101110', '01100011', '01110010', '01111001', '01110000', '01110100', '01101001', '01101111', '01101110', '00